In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pwd

/content


In [ ]:
WORKING_PATH = '/content/drive/MyDrive/NLP'

In [ ]:
%cd {WORKING_PATH}

/content/drive/MyDrive/NLP


In [ ]:
!ls

'1. Introduccion.pdf'				        requirements.txt
'2.Introduccion al Procesado de Lenguaje Natural.pdf'   sent.txt
 Clases						        Soluciones
 Datasets					        utils.py
 datos_prepocesados.csv				        w2v_model_fast.pkl
'Práctica final NLP.pdf'			        w2v_model_fast.pkl.wv.vectors_ngrams.npy
 Proyecto_final					        w2v_model.pkl
 __pycache__


In [ ]:
!pip install unzip
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1283 sha256=3dc27b20f9d811eb01848b16ecab7b70217974ba647c50e782446a3d0128e553
  Stored in directory: /root/.cache/pip/wheels/3c/4d/b3/ddd83a91322fba02a91898d3b006090d1df1d3b0ad61bd8b36
Successfully built unzip
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
  Created wheel for pyDAWG: filename=pyDAWG-1.0.1-cp311-cp311-linux_x86_64.whl size=62786 sha256=dd40d6e9470f873e3486a8be82893988e04d15fc3d5b64b7f35af3376f6000c3
  Stored in directory: /root/.cache/pip/wheels/b3/42/a5/2904aff781887b41600fdaf29aac

In [ ]:
import pandas as pd  # Importa pandas

df = pd.read_json('./Proyecto_final/All_Beauty_5.json', lines=True)



In [ ]:
# libreria para conertir de numeros a palabras
!pip install inflect
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import re
import inflect
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import contractions
import spacy

#Descargar y cargar el modelo de spaCy
nlp = spacy.load("en_core_web_sm")

#Corregir errores ortográficos
def corregir_ortografia(texto):
    blob = TextBlob(texto)
    return str(blob.correct())

#Expandir contracciones comunes en inglés
def expandir_contracciones(texto):
    return contractions.fix(texto)




#Funcion para convertir número a plabras
def convertir_numeros_a_palabras(texto):
    p = inflect.engine()
    palabras = texto.split()
    palabras_convertidas = []
    for palabra in palabras:
        if palabra.isdigit():
            palabra = p.number_to_words(palabra)
        palabras_convertidas.append(palabra)

    return ' '.join(palabras_convertidas)

# Función para convertir a minúsculas
def convertir_minusculas(texto):
    return texto.lower()

# Función para eliminar signos de puntuación
def eliminar_puntuacion(texto):
    return re.sub(r'[^\w\s]', '', texto)


# Función de preprocesamiento principal
def preprocesar_texto_spacy(texto):
    if isinstance(texto, float):  # Manejar NaN o valores no cadena
        return ""

    texto = texto.strip() #Eliminamos espacios al principio y al final
    texto = convertir_minusculas(texto)
    texto = corregir_ortografia(texto)
    texto = expandir_contracciones(texto)
    texto = convertir_numeros_a_palabras(texto)
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = eliminar_puntuacion(texto)

    doc = nlp(texto)

    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return ' '.join(tokens)

# Verificar valores NaN en 'reviewText'
print("Valores NaN en 'reviewText':\n", df['reviewText'].isna().sum())

# Convertir NaN a cadenas vacías en la columna 'reviewText'
df['reviewText'] = df['reviewText'].fillna('')

# Aplicar preprocesamiento a la columna 'reviewText'
df['reviewTextProcesado'] = df['reviewText'].apply(preprocesar_texto_spacy)

# Verificar si la columna fue creada correctamente
print("\nColumnas en DataFrame:\n", df.columns)

# Mostrar algunas filas del DataFrame para verificar el resultado
print(df[['reviewText', 'reviewTextProcesado']].head())

#Guardar el DataFrame en Drive
ruta_archivo = '/content/drive/MyDrive/NLP/datos_prepocesados.csv'
df.to_csv(ruta_archivo, index=False)

print(f'DataFrame guardado en:{ruta_archivo}]')


Valores NaN en 'reviewText':
 5

Columnas en DataFrame:
 Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'reviewTextProcesado'],
      dtype='object')
                                          reviewText  \
0                   As advertised. Reasonably priced   
1  Like the oder and the feel when I put it on my...   
2  I bought this to smell nice after I shave.  Wh...   
3  HEY!! I am an Aqua Velva Man and absolutely lo...   
4  If you ever want to feel pampered by a shampoo...   

                                 reviewTextProcesado  
0                         advertise reasonably price  
1  like oder feel face try band review people kno...  
2  buy smell nice shave smell awful nineteen smel...  
3  hey anna vulva man absolutely love stuff year ...  
4  want feel pamper shampoo smell like wonderful ...  
DataFrame guardado en:/content/drive/MyDrive/NLP/datos_prepocesad

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP/datos_prepocesados.csv")
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,reviewTextProcesado
0,5,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN,advertise reasonably price
1,5,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN,like oder feel face try band review people kno...
2,1,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN,buy smell nice shave smell awful nineteen smel...
3,5,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN,hey anna vulva man absolutely love stuff year ...
4,5,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN,want feel pamper shampoo smell like wonderful ...
